# HIV Analysis

Here we combine HIV sequence data from the [Los Alamos National Laboratory HIV Sequence Database](www.hiv.lanl.gov) and immunological data to investigate HIV evolution across 14 individuals. This data is contained in the `data/HIV/` directory. Supplementary scripts for data processing are stored in `HIV.py`.

We downloaded sequence data for all patients and aligned them with the [HXB2 reference sequence](https://www.hiv.lanl.gov/components/sequence/HIV/asearch/query_one.comp?se_id=K03455) and the corresponding [clade consensus sequence](https://www.hiv.lanl.gov/content/sequence/NEWALIGN/align.html#consensus) using the LANL tool [HIValign](https://www.hiv.lanl.gov/content/sequence/VIRALIGN/viralign.html). We recorded immunological data for these patients in `.csv` files for analysis.

### Sequence processing

In the scripts below, we first process the HIV sequence data by
1. associating times with sequences and removing all sequences for which no collection time is available,
2. clipping full-length sequences down to the heavily-sequenced half-genome regions,
3. removing sequences with excess gaps in the targeted region, which primarily consist of sequences that cover the opposite half-genome region,
4. removing sites that are >= 95% gaps,
5. time-ordering sequences,
6. removing time points that are > 300 days from the previous time point and/or have < 4 sequences available at that time point, in order to ensure minimum quality of the correlations,
6. imputing ambiguous nucleotides with the most frequently observed nucleotide at the same site, and
7. saving the data in a format readable by the MPL program.

During this process we track the location of known T cell epitopes and exposed regions of Env, and we record the transmitted/founder and clade consensus nucleotides at each site. We also save a shell script `HIV/MPL/run.sh` for running MPL to infer the selection coefficients underlying each trajectory.

### Load libraries and define global variables

In [1]:
print('This notebook was prepared using:')

import os
import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import HIV

# GLOBAL VARIABLES

NUC = ['-', 'A', 'C', 'G', 'T']
PRO = ['-', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
       'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
REF = NUC[0]
CONS_TAG = 'CONSENSUS'
HXB2_TAG = 'B.FR.1983.HXB2-LAI-IIIB-BRU.K03455.19535'
HIV_DIR  = 'data/HIV'
MPL_DIR  = 'src/MPL'

This notebook was prepared using:
python version 3.7.3 (default, Mar 27 2019, 09:23:15) 
[Clang 10.0.1 (clang-1001.0.46.3)]
numpy version 1.16.4
pandas version 0.24.2


### Process sequences

In [2]:
# Load the sequence coverage for each patient
df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the list of transmitted/founder (TF) sequences
df_TF = pd.read_csv('%s/TF.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the list of exposed residues on Env
df_exposed = pd.read_csv('%s/exposed.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the set of epitopes targeted by patients
df_epitope = pd.read_csv('%s/epitopes.csv' % (HIV_DIR), comment='#', dtype={'ppt': str}, memory_map=True)


# Association times with sequences, remove all sequences (except HXB2 and consensus)
# that do not have identifiable times; TF sequence is treated as day 0 if available
for ppt in set(df_range.ppt):
    HIV.filter_sequences(ppt, '%s/raw/%s-HIValign.fasta' % (HIV_DIR, ppt), check_time=True)
    
    
# Store baseline number of nonsynonymous mutations in CD8+ T cell epitopes, 
# nonsynonymous reversions (total, in epitopes, and outside epitopes), and synonymous mutations
BASELINE_NS_EPITOPE = 0
BASELINE_NS_REV = 0
BASELINE_NS_REV_NONEPI = 0
BASELINE_SYN = 0
BASELINE_TOT = 0

# Select and process alignment subsets for analysis
g = open('%s/run.sh' % (MPL_DIR), 'w')
g.write('g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl\n')

for df_range_iter, df_range_entry in df_range.iterrows():
    
    # Read in MSA
    ppt = df_range_entry.ppt
    msa, tag = HIV.get_MSA('%s/interim/%s-HIValign-filtered.fasta' % (HIV_DIR, ppt), noArrow=True)
    print('%s' % df_range_entry.tag)
    
    # Clip MSA to specified range
    msa = HIV.clip_MSA(df_range_entry.start, df_range_entry.end, msa, tag)
    
    # Filter sequences for quality (maximum # excess gaps)
    max_gap_num = 200
    max_gap_freq = 0.05
    current_msa, current_tag = HIV.filter_excess_gaps(msa, tag, max_gap_num, max_gap_freq, verbose=True)
    
    # Put sequences in time order (first entries are HXB2, consensus)
    current_msa, current_tag = HIV.order_sequences(current_msa, current_tag)
    
    # Locate or compute TF sequence
    TF_accession = df_TF[df_TF.ppt==ppt].iloc[0].TF
    TF_sequence = HIV.get_TF(current_msa, current_tag, TF_accession)
    
    temp_msa = [current_msa[list(current_tag).index(HXB2_TAG)], TF_sequence]
    temp_tag = [HXB2_TAG, '%s-TF' % ppt]
    HIV.save_MSA(temp_msa, temp_tag, '%s/interim/TF-%s' % (HIV_DIR, df_range_entry.tag))
    
    # Impute ambiguous nucleotides and gaps at start/end of alignment
    current_msa = HIV.impute_ambiguous(current_msa, current_tag, start_index=2, verbose=True)
    
    # Map between local (alignment) index, index of polymorphic sites, and HXB2
    # Track location of known T cell epitopes, TF sequence, and exposed residues of Env
    min_seqs = 4
    max_dt = 300
    poly_sites, current_msa, current_tag = HIV.create_index(current_msa, current_tag, TF_sequence, 
                                           current_msa[list(current_tag).index(CONS_TAG)], 
                                           current_msa[list(current_tag).index(HXB2_TAG)],
                                           df_range_entry.start, min_seqs, max_dt, 
                                           df_epitope[df_epitope.ppt==ppt], df_exposed, 
                                           '%s/processed/%s-index.csv' % (HIV_DIR, df_range_entry.tag), 
                                           return_polymorphic=True, return_truncated=True)
    
    # Save alignment to format readable by MPL
    poly_states, poly_times = HIV.save_MPL_alignment(current_msa, current_tag, 
                                                     '%s/HIV/%s-poly-seq2state.dat' % (MPL_DIR, df_range_entry.tag), 
                                                     polymorphic_sites=poly_sites, return_states=True)
    
    # Save allele frequency trajectories for polymorphic sites
    # Later this file will be edited to include inferred selection coefficients
    df_index = pd.read_csv('%s/processed/%s-index.csv' % (HIV_DIR, df_range_entry.tag), comment='#', memory_map=True)
    HIV.save_trajectories(poly_sites, poly_states, poly_times, TF_sequence, df_index,
                          '%s/interim/%s-poly.csv' % (HIV_DIR, df_range_entry.tag))
    
    # Get baseline number of possible mutations in different categories
    if ppt not in ['cap256']:
        df_poly = pd.read_csv('%s/interim/%s-poly.csv' % (HIV_DIR, df_range_entry.tag), comment='#', memory_map=True)
        temp_ns_epitope, temp_ns_rev, temp_ns_rev_ne, temp_syn = HIV.get_baseline(TF_sequence, df_index, df_poly)
        BASELINE_NS_EPITOPE += temp_ns_epitope
        BASELINE_NS_REV += temp_ns_rev
        BASELINE_NS_REV_NONEPI += temp_ns_rev_ne
        BASELINE_SYN += temp_syn
        BASELINE_TOT += 3 * len(df_index) + np.sum(df_poly.nucleotide=='-')
    
    # Save MSA
    HIV.save_MSA(current_msa, current_tag, '%s/interim/%s' % (HIV_DIR, df_range_entry.tag))
    
    # Record instructions for running MPL on this data
    io_str = '%s-poly-seq2state' % df_range_entry.tag
    run_str = ('./bin/mpl -d HIV -i %s.dat -o %s-MPL.dat -g 5e4 -N 1e4 -m Zanini-extended.dat -sc %s -sn %s\n'+
               './bin/mpl -d HIV -i %s.dat -o %s-SL.dat  -g 5e4 -N 1e4 -m Zanini-extended.dat -nc\n')
    g.write(run_str % (io_str, io_str, 'covariance-'+io_str+'.dat', 'numerator-'+io_str+'.dat', io_str, io_str))

g.close()


# Print baseline numbers of possible mutations in different categories
print('\nOut of %d possible mutations,' % BASELINE_TOT)
print('\t%d (%.2f%%) nonsynonymous in epitope' 
      % (BASELINE_NS_EPITOPE, 100*BASELINE_NS_EPITOPE/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion'
      % (BASELINE_NS_REV, 100*BASELINE_NS_REV/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion, in epitope' 
      % (BASELINE_NS_REV-BASELINE_NS_REV_NONEPI, 100*(BASELINE_NS_REV-BASELINE_NS_REV_NONEPI)/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion, nonepitope' 
      % (BASELINE_NS_REV_NONEPI, 100*BASELINE_NS_REV_NONEPI/BASELINE_TOT))
print('\t%d (%.2f%%) synonymous'
      % (BASELINE_SYN, 100*BASELINE_SYN/BASELINE_TOT))

706010164-5
	selected 98 of 199 sequences with <200 gaps in excess of consensus
	removed 2 of 4262 sites with >95% gaps
	exchanged A for R in sequence 72, site 97
	exchanged T for Y in sequence 37, site 224
	exchanged C for Y in sequence 37, site 343
	exchanged A for R in sequence 40, site 638
	exchanged A for N in sequence 97, site 693
	exchanged G for R in sequence 3, site 747
	exchanged G for R in sequence 14, site 747
	exchanged G for R in sequence 22, site 747
	exchanged G for R in sequence 28, site 747
	exchanged G for S in sequence 37, site 897
	exchanged C for Y in sequence 66, site 1011
	exchanged C for Y in sequence 65, site 1112
	exchanged A for R in sequence 63, site 1119
	exchanged C for Y in sequence 3, site 1190
	exchanged A for R in sequence 4, site 1382
	exchanged A for R in sequence 45, site 1491
	exchanged T for Y in sequence 32, site 1646
	exchanged A for R in sequence 44, site 2136
	exchanged A for R in sequence 34, site 2302
	exchanged T for N in sequence 88, site

	exchanged T for W in sequence 17, site 3565
	exchanged T for Y in sequence 49, site 3830
	exchanged G for R in sequence 82, site 4017
	exchanged A for R in sequence 45, site 4045
	mutant at site 4253 in codon that does not terminate in alignment, assuming syn
	mutant at site 4253 in codon that does not terminate in alignment, assuming syn
704010042-3
	selected 93 of 179 sequences with <200 gaps in excess of consensus
	removed 62 of 4714 sites with >95% gaps
	exchanged A for R in sequence 37, site 579
	exchanged T for W in sequence 14, site 767
	exchanged T for Y in sequence 23, site 899
	exchanged A for R in sequence 48, site 1080
	exchanged C for Y in sequence 14, site 1092
	exchanged T for Y in sequence 44, site 1377
	exchanged G for R in sequence 14, site 2530
	exchanged A for R in sequence 69, site 2764
	exchanged A for R in sequence 10, site 2896
	exchanged T for Y in sequence 42, site 3463
	exchanged A for R in sequence 16, site 3617
	exchanged T for Y in sequence 44, site 3685


### Add superinfection sequence (SU) to index CSV for CAP256

In [3]:
# Load the sequence coverage for each patient
df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)

# Read in MSA
ppt = 'cap256'
HIV.filter_sequences(ppt+'-SU', '%s/raw/%s-HIValign.fasta' % (HIV_DIR, ppt+'-SU'), check_time=True)
msa, tag = HIV.get_MSA('%s/interim/%s-HIValign-filtered.fasta' % (HIV_DIR, ppt+'-SU'), noArrow=True)

# Clip MSA to specified range
df_range = df_range[df_range.ppt=='cap256']
msa = HIV.clip_MSA(df_range.iloc[0].start, df_range.iloc[0].end, msa, tag)

# Filter sequences for quality (maximum # excess gaps)
max_gap_num = 200
max_gap_freq = 0.05
current_msa, current_tag = HIV.filter_excess_gaps(msa, tag, max_gap_num, max_gap_freq, verbose=True)
    
# Put sequences in time order (first entries are HXB2, consensus)
current_msa, current_tag = HIV.order_sequences(current_msa, current_tag)
    
# Compute SU sequence
SU_accession = 'avg'
SU_sequence  = HIV.get_TF(current_msa, current_tag, SU_accession)

# Add SU sequence to index CSV
df_index = pd.read_csv('%s/processed/%s-3-index.csv' % (HIV_DIR, ppt), comment='#', memory_map=True)
df_index = df_index.assign(SU=pd.Series(list(SU_sequence)))
df_index.to_csv('%s/processed/%s-3-index.csv' % (HIV_DIR, ppt+'-SU'), index=False)

	selected 168 of 183 sequences with <200 gaps in excess of consensus
	removed 5 of 2651 sites with >95% gaps


### Shift selection coefficients

After selection has been inferred, the selection coefficients should be normalized such that the TF nucleotide at each site has a selection coefficient of zero. The cell below performs this normalization and saves all the selection coefficients together in the file `HIV/analysis/total-selection.csv`.

In [2]:
# Shift selection coefficients

df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)
df_epitope = pd.read_csv('%s/epitopes.csv' % (HIV_DIR), comment='#', dtype={'ppt': str}, memory_map=True)

f = open('%s/analysis/total-selection.csv' % (HIV_DIR), 'w')
f.write('ppt,tag,polymorphic_index,alignment_index,HXB2_index,consensus_nucleotide,TF_nucleotide,nucleotide,' +
        'in_epitope,epitope,epitope_detected,flanking,exposed,glycan,nonsynonymous,s_MPL,s_SL\n')

# potential future comparisons: set point, clade, time epitope targeted, immunodominance, M/F, age

for idx, entry in df_range.iterrows():
    ppt = entry.ppt
    
    # Read in MPL output
    temp_MPL = np.loadtxt('%s/HIV/%s-poly-seq2state-MPL.dat' % (MPL_DIR, entry.tag))
    temp_SL = np.loadtxt('%s/HIV/%s-poly-seq2state-SL.dat' % (MPL_DIR, entry.tag))
    L = int(len(temp_MPL)/len(NUC))
    temp_MPL = np.reshape(temp_MPL, (L, len(NUC)))
    temp_SL = np.reshape(temp_SL, (L, len(NUC)))
    
    df_info = pd.read_csv('%s/interim/%s-poly.csv' % (HIV_DIR, entry.tag), comment='#', memory_map=True)
    
    # Normalize selection coefficients
    s_MPL = []
    s_SL = []
    for i in range(L):
        TF = df_info[df_info.polymorphic_index==i].iloc[0].TF
        idx = NUC.index(TF)
        
        temp_s = temp_MPL[i]
        temp_s = temp_s - temp_s[idx]
        s_MPL.append(temp_s)
    
        temp_s = temp_SL[i]
        temp_s = temp_s - temp_s[idx]
        s_SL.append(temp_s)
    
    reduced_s_MPL = []
    reduced_s_SL = []
    for df_info_iter, df_info_entry in df_info.iterrows():
        i = df_info_entry.polymorphic_index
        j = NUC.index(df_info_entry.nucleotide)
        reduced_s_MPL.append(s_MPL[i][j])
        reduced_s_SL.append(s_SL[i][j])
    
    # Record data
    df_info['s_MPL'] = pd.Series(reduced_s_MPL, index=df_info.index)
    df_info['s_SL'] = pd.Series(reduced_s_SL, index=df_info.index)
    
    df_info.to_csv('%s/analysis/%s-poly.csv' % (HIV_DIR, entry.tag), index=False)
    
    mean_MPL = np.mean(df_info.s_MPL)
    std_MPL = np.std(df_info.s_MPL)
    mean_SL = np.mean(df_info.s_SL)
    std_SL = np.std(df_info.s_SL)
    
    for df_info_iter, df_info_entry in df_info.iterrows():
        if df_info_entry.TF==df_info_entry.nucleotide:
            continue
        detected = 'nan'
        if pd.notnull(df_info_entry.epitope):
            detected = str(int(df_epitope[((df_epitope.ppt==ppt) & 
                                           (df_epitope.epitope==df_info_entry.epitope))].detected))
        f.write('%s,%s,%s,%d,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%.4f,%.4f\n' % 
                (ppt, entry.tag, df_info_entry.polymorphic_index, df_info_entry.alignment_index, 
                 df_info_entry.HXB2_index, df_info_entry.consensus, df_info_entry.TF,
                 df_info_entry.nucleotide, not pd.isnull(df_info_entry.epitope), df_info_entry.epitope, 
                 detected, str(int(df_info_entry.flanking)), df_info_entry.exposed,
                 str(int(df_info_entry.glycan)), str(int(df_info_entry.nonsynonymous)), 
                 df_info_entry.s_MPL, df_info_entry.s_SL))

f.close()    

### Quantify influence of individual variants on inferred selection at other sites

The selection coefficients that we infer depend not only on the trajectories of individual variants, but also on the genetic background(s) on which they arise. To quantify this, we generate *masked* trajectories where individual variants are replaced by the TF nucleotide. We can then compare selection inferred with and without masking to determine how each variant affects estimates of selection for other variants. In the cells below we generate and save the masked trajectories, then we normalize the and compare the inferred selection with and without masking.

In [ ]:
# Generate masked trajectories and associated job files for MPL

f = open('%s/jobs/run-ds.sh' % (MPL_DIR), 'w')

df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)
for it, entry in df_range.iterrows():
    tag = entry.tag
    df = pd.read_csv('%s/analysis/%s-poly.csv' % (HIV_DIR, tag), comment='#', memory_map=True)
    df_noTF = df[df.nucleotide!=df.TF]

    # Get sequence data
    traj = np.loadtxt('%s/HIV/%s-poly-seq2state.dat' % (MPL_DIR, tag))
    
    for var_it, var_entry in df_noTF.iterrows():
        var_id = str(int(var_entry.polymorphic_index))+var_entry.nucleotide
        
        # Save masked trajectory
        temp_traj = np.copy(traj)
        poly_idxs = traj.T[var_entry.polymorphic_index+2]==NUC.index(var_entry.nucleotide)
        temp_traj.T[var_entry.polymorphic_index+2][poly_idxs] = NUC.index(var_entry.TF)
        np.savetxt('%s/HIV/mask/%s-poly-seq2state-%s.dat' % (MPL_DIR, tag, var_id), temp_traj, fmt='%d')
    
        # Write job to file
        f.write('./bin/mpl -d HIV/mask -i %s-poly-seq2state-%s.dat -o %s-poly-seq2state-MPL-%s.dat' 
                % (tag, var_id, tag, var_id))
        f.write(' -g 5e4 -N 1e4 -m Zanini-extended.dat\n')

f.close()

In [ ]:
# Shift and compare selection coefficients

df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)
for it, entry in df_range.iterrows():
    
    # Read in information on polymorphic sites, TF, selection
    tag = entry.tag
    df = pd.read_csv('%s/analysis/%s-poly.csv' % (HIV_DIR, tag), comment='#', memory_map=True)
    L = len(np.unique(df.polymorphic_index))
    TF_idxs = [NUC.index(df[df.polymorphic_index==i].iloc[0].TF) for i in range(L)]
    
    f = open('%s/analysis/%s-delta-s.csv' % (HIV_DIR, tag), 'w')
    f.write('mask_polymorphic_index,mask_nucleotide,target_polymorphic_index,target_nucleotide,effect,distance\n')
    
    # Iterate through masked variants
    df_noTF = df[df.nucleotide!=df.TF]
    for var_it, var_entry in df_noTF.iterrows():
        var_id = str(int(var_entry.polymorphic_index))+var_entry.nucleotide
        
        temp_MPL = np.loadtxt('%s/HIV/mask/%s-poly-seq2state-MPL-%s.dat' % (MPL_DIR, entry.tag, var_id))
        temp_MPL = np.reshape(temp_MPL, (L, len(NUC)))
        mask_MPL = [temp_MPL[i] - temp_MPL[i][TF_idxs[i]] for i in range(L)]
        
        for target_it, target_entry in df_noTF.iterrows():
            f.write('%d,%s,%d,%s,' % (int(var_entry.polymorphic_index), var_entry.nucleotide, 
                                      int(target_entry.polymorphic_index), target_entry.nucleotide))
            f.write('%le,' % (target_entry.s_MPL 
                              - mask_MPL[int(target_entry.polymorphic_index)][NUC.index(target_entry.nucleotide)]))
            f.write('%d\n' % (np.fabs(int(var_entry.alignment_index) - int(target_entry.alignment_index))))
    
    f.close()